## Preprocessing
https://notebook.community/fschueler/incubator-systemml/projects/breast_cancer/Preprocessing

## Setup

In [239]:
import math
import os
import matplotlib.pyplot as plt
import numpy as np
import openslide
from openslide.deepzoom import DeepZoomGenerator
import pandas as pd
# from pyspark.mllib.linalg import Vectors
from scipy.ndimage.morphology import binary_fill_holes
from skimage.color import rgb2gray
from skimage.feature import canny
from skimage.morphology import binary_closing, binary_dilation, disk

from PIL import Image
# from tqdm import tqdm
from tqdm.notebook import tqdm

plt.rcParams['figure.figsize'] = (10, 6)

## Open Whole-Slide Image

In [240]:
def open_slide(slide_num, folder):
    '''画像番号を指定してWSIを開く
    :param slide_num: スライド番号
    :param folder: スライドフォルダが入っているディレクトリ
  
    :Return OpenSlide objectのWSI
    '''
    slide_names = os.listdir(folder)
    filename = os.path.join(folder, slide_names[slide_num])
 
    slide = openslide.open_slide(filename)
    return slide

## Create Tile Generator

In [241]:
def create_tile_generator(slide, tile_size=1024, overlap=0):
    '''WSIからタイル画像を抽出するためのgeneratorを生成する

    :param slide: OpenSlide objectのWSI
    :param tile_size: 生成されるタイル画像ｇあの幅と高さを指定（正方形）
    :param overlap: タイル間のオーバーラップのピクセル数

    :Returns generator
    Note: 抽出されたタイル画像はShape(tile_size, tile_size, channels)を持つ
    '''
    generator = DeepZoomGenerator(slide, tile_size=tile_size, overlap=overlap, limit_bounds=True)
    return generator

## Determine 20x Magnification Zoom Level

In [242]:
def get_20x_zoom_level(slide, generator):
    '''20倍に相当するzoom levelを返す
    generatorは複数ズームレベルからタイルを抽出
    高解像度から低解像度までの各レベルで2倍のダウンサンプリングをする

    param: slide: OpenSlide objectのWSI
    param: generator: DeepZoomGenerator object

    Return: 20倍に相当するズームレベルまたはそれに近いレベル
    '''
    # level_count - 1とすることで最低解像度のレベルindexを取得
    highest_zoom_level = generator.level_count - 1  # 0-based indexing

    try:
        # スライドの対物レンズの倍率を表すプロパティを取得
        mag = int(slide.properties[openslide.PROPERTY_NAME_OBJECTIVE_POWER])
        
        # mag / 20: スライドの倍率と望んだ倍率との間のダウンサンプリング係数
        # (mag / 20) / 2: generatorのダウンサンプリング係数に基づいて、最高解像度レベルからのズームレベルのオフセット
        offset = math.floor((mag / 20) / 2)

        level = highest_zoom_level - offset

    except ValueError:
        # スライドの倍率がわからない場合は最高解像度を返す
        level = highest_zoom_level
        
    return level

## Generate Tile Indices For Whole-Slide Image.

In [243]:
def process_slide(slide_num, folder, tile_size=1024, overlap=0):
    '''WSIに対して可能な全てのタイル画像のインデックスを生成
    を生成

    :param slide_num: スライド番号
    :param folder: スライドフォルダが入っているディレクトリ
    :param tile_size: 生成されるタイル画像の幅と高さを指定（正方形）
    overlap: タイル間のオーバーラップのピクセル数

    Return: (slide_num, tile_size, overlap, zoom_level, col, row)のリスト
    '''
    # WSIを開く
    slide = open_slide(slide_num, folder)

    # generatorを生成
    generator = create_tile_generator(slide, tile_size, overlap)

    # 20倍のzoom levelを取得
    zoom_level = get_20x_zoom_level(slide, generator)

    # 可能な全てのタイル画像のインデックスを生成
    # 指定したレベルの(tile_x, tile_y)タプルのリストを返す
    cols, rows = generator.level_tiles[zoom_level]

    tile_indices = [(slide_num, tile_size, overlap, zoom_level, col, row)
                    for col in range(cols) for row in range(rows)]

    return tile_indices

## Generate Tile From Tile Index

In [244]:
def process_tile_index(tile_index, folder):
    '''タイルインデックスからタイル画像を生成
    :param tile_index: 抽出するタイルを表すインデックスタプル
    :param folder: スライドフォルダが入っているディレクトリ
     
    Return: (slide_num, tile)のタプル
            RGB形式の3次元Numpy配列 (tile_size, tile_size, channels)
    '''
    slide_num, tile_size, overlap, zoom_level, col, row = tile_index
    
    # WSIを開く
    slide = open_slide(slide_num, folder)

    # generatorを生成
    generator = create_tile_generator(slide, tile_size, overlap)

    # タイル画像を生成
    tile = np.array(generator.get_tile(zoom_level, (col, row)))
    
    return (slide_num, tile)

## Filter Tile For Dimensions & Tissue Threshold

In [245]:
def keep_tile(tile_tuple, tile_size=1024, tissue_threshold=0.9):
    '''タイル画像を残すかどうか判断する
    サイズと組織割合の閾値に基づいてタイル画像をフィルタリングする
    あるタイル画像の高さと幅が(tile_size, tile_size)に等しく、
    かつ、指定された％以上の成分を含むタイルを保持する
    それ以外の場合はフィルタリングされる

    param: tile_tuple: (slide_num, tile) のタプル
    param: tile_size: 生成されるタイル画像の幅と高さを指定（正方形）
    param: tissue_threshold: 組織割合の閾値

    Return: タイル画像を保持するかどうかのブール値
    '''
    slide_num, tile = tile_tuple
    
    if tile.shape[0:2] == (tile_size, tile_size):
        # 3DのRGB画像を2Dのgrayscaleに変換
        # 0 (高密度) から1 (無地の背景)
        tile = rgb2gray(tile)
        
        # 8-bitのDepthを補完 1から0まで
        tile = 1 - tile

        # hysteresis thresholdingによるCanny edgeの検出
        # 1がedgeに相当（組織にはedgeが多い、背景はそうでもないということ）
        tile = canny(tile)

        # Binary closing
        # 拡張の後に収縮を行うことで背景のノイズを取り除く
        tile = binary_closing(tile, disk(10))

        # Binary dilation
        # 明るい部分を拡大し、暗い部分を縮小することで組織領域内の穴を埋める
        tile = binary_dilation(tile, disk(10))

        # 組織領域内の残りの穴を埋める
        tile = binary_fill_holes(tile)

        # 組織のカバー率を算出
        percentage = tile.mean()
        
        return percentage >= tissue_threshold
    
    else:
      return False

## Generate Flattened Samples From Tile

In [246]:
def process_tile(tile_tuple, sample_size=256, grayscale=False):
    '''タイル画像をより小さな画像に加工する
    タイル画像をsample_size * sample_sizeピクセルの小さなタイルに切り分ける
    各サンプルの形状を(H, W, C)から(C, H, W)に変換する
    それぞれを長さC * H * Wの長さのベクトルに変換

    param: tile_tuple: (slide_num, tile) のタプル
    param: sample_size: 生成されるタイル画像の幅と高さを指定（正方形）
    grayscale: RGBではなくgrayscaleのサンプルを生成するかどうか。

    Return: Flattenされた(channels * sample_size_x * sample_size_y)のベクトル
    '''
    slide_num, tile = tile_tuple

    if grayscale:
      # grayscaleに変換
      tile = rgb2gray(tile)[:, :, np.newaxis]
      
      # [0, 1]から[0, 255]に変換することでディスク容量と時間を節約できるが、若干の情報破損が生じる
      tile = np.round(tile * 255).astype("uint8")

    x, y, ch = tile.shape
    
    # (num_x, sample_size_x, num_y, sample_size_y, ch）の5次元配列にreshape
    # num_x, yはそれぞれx軸とy軸に分割されたタイルの数
    samples = (tile.reshape((x // sample_size, sample_size, y // sample_size, sample_size, ch))
                    .swapaxes(1,2) # sample_size_xとnum_yの軸を入れ替え (num_x, num_y, sample_size_x, sample_size_y, ch)
                    .reshape((-1, sample_size, sample_size, ch)) # num_xとnum_yを1つの軸に結合 (num_samples, sample_size_x, sample_size_y, ch)
                    .transpose(0,3,1,2)) # (num_samples, ch, sample_size_x, sample_size_y)

    # Flatten (num_samples, ch * sample_size_x * sample_size_y)
    samples = samples.reshape(samples.shape[0], -1)

    samples = [(slide_num, sample) for sample in list(samples)]

    return samples

## Visualize Tile

In [247]:
def visualize_tile(tile):
    '''タイル画像をプロットする
    param: tile: 3次元のNumpy配列 (tile_size, tile_size, channels)

    Return: None
    '''
    plt.imshow(tile)
    plt.show()

## Visualize Sample

In [248]:
def visualize_sample(sample, size=256):
    '''画像サンプルをプロットする
    param: sample: 正方形の画像をflattenしたvectors (channels * size_x * size_y)
    param: size: 正方形の画像の幅と高さ

    Return: None
    '''
    # (size_x, size_y, channels)へ変換
    length = sample.shape[0]
    channels = int(length / (size * size))

    if channels > 1:
        sample = sample.astype('uint8').reshape((channels, size, size)).transpose(1,2,0)
        plt.imshow(sample)
    else:
        vmax = 255 if sample.max() > 1 else 1
        sample = sample.reshape((size, size))
        plt.imshow(sample, cmap="gray", vmin=0, vmax=vmax)
        plt.show()

In [249]:
def get_file_name(slide_num, folder):
    '''ファイル名を取得
    '''
    slide_names = os.listdir(folder)
    filename = slide_names[slide_num].split('.')

    # 拡張子はいらない
    return filename[0]

In [250]:
def make_dirs(output_dir, slide_name):
    '''保存用のディレクトリを作成
    '''
    if not os.path.exists(f'{output_dir + slide_name}'):
        os.mkdir(f'{output_dir + slide_name}')

In [260]:
def save_jpeg_images(sample, sample_num, slide_name):
    '''flattenされたvectorから画像をjpegで保存
    :param sample: (channels * sample_size_x * sample_size_y)
    '''
    # 画像長を抽出し、チャネル数を算出
    length = sample.shape[0]
    channels = int(length / (256 * 256))

    # 配列を(H*W*C)の形式に変換
    image = sample.astype('uint8').reshape((channels, 256, 256)).transpose(1,2,0)

    # PILフォーマットに変換
    pil_image = Image.fromarray(image)

    # 保存 WSI名_連番の形式で
    pil_image.save(f'../output/{slide_name}/{slide_name}_{sample_num}.jpeg')

## Process All Slides And Save

In [261]:
def process(slide_num, folder=slide_dir, output_dir=output_dir, tile_size=1024, over_lap=0, tissue_threshold=0.9, sample_size=256, grayscale=False):
    print('Process start....')
    # WSIに対して可能な全てのタイル画像のインデックスを生成
    tile_idx = process_slide(slide_num, folder, tile_size, over_lap)

    print('Generate tiled image from index....')
    # タイルインデックスからタイル画像を生成
    tiles = [process_tile_index(i, folder) for i in tqdm(tile_idx)]

    print('Filtering a tile image....')
    # タイル画像のフィルタリング
    filtered_tiles = list(filter(lambda tile: keep_tile(tile, tile_size, tissue_threshold), tqdm(tiles)))

    # タイル画像をより小さなタイル画像にする
    samples = [n for i in filtered_tiles for n in process_tile(i, sample_size, grayscale)]

    # ファイル名を取得
    slide_name = get_file_name(slide_num, folder)

    # 保存用のディレクトリを作成
    make_dirs(output_dir, slide_name)

    print('Saving Tile Images....')
    # flatten vectorから画像をPILフォーマットに変換し保存
    for sample_num, sample in enumerate(tqdm(samples)):
        sample = sample[1]
        save_jpeg_images(sample, sample_num, slide_name)


In [264]:
slide_dir = '../input/'
output_dir = '../output/'

In [265]:
process(2)

Process start....
Generate tiled image from index....



Filtering a tile image....



Saving Tile Images....
